In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_14545/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [61]:
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import folium.plugins
from itertools import product
from pyproj import Geod
wgs84_geod = Geod(ellps='WGS84') # Distance will be measured in meters on this ellipsoid - more accurate than a spherical method
from tqdm import tqdm

In [4]:
actions = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'actions.csv'), parse_dates=['Time'])

In [15]:
for _, action in actions[actions['Time']==datetime(2020, 2, 2, 9, 30, 0)].iterrows():
    print(action[action==1].index[action[action==1].index.str.contains('Vehicle')][0][14:])

BMW_1ER


In [62]:
area_centers = pd.read_csv((Path.cwd().parent / 'data' / 'processed' / 'areas.csv'), index_col=0)
rental = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'rental.csv'), parse_dates=['Start_Datetime_Local', 'End_Datetime_Local'])
demand = pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'openings.csv'), parse_dates=['Created_Datetime_Local'])

In [63]:
rental=pd.read_csv((Path.cwd().parent / 'data' / 'interim' / 'rental.csv'), parse_dates=['Start_Datetime_Local', 'End_Datetime_Local'])

In [10]:
dists = np.zeros([len(area_centers), len(area_centers)])
for o, d in list(product(area_centers.index, repeat=2)):
    _,_,dists[o,d] = wgs84_geod.inv(
        area_centers.iloc[o]['GPS_Longitude'], area_centers.iloc[o]['GPS_Latitude'],
        area_centers.iloc[d]['GPS_Longitude'], area_centers.iloc[d]['GPS_Latitude'])

In [47]:
d = 0
for _, r in tqdm(rental.iterrows()):
    d = d + dists[r['Virtual_Start_Zone_Name'],r['Virtual_End_Zone_Name']]

1594902it [01:57, 13516.12it/s]


In [50]:
dist_per_trip = d/len(rental)

In [64]:
rental['Revenue_Net'].sum()

113588545.53999996

In [54]:
rental['Revenue_Net'].sum()/dist_per_trip

28985.680888589708

In [60]:
((rental['End_Datetime_Local']-rental['Start_Datetime_Local']).mean()/dist_per_trip).total_seconds()

0.846283

In [23]:
start_map = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
for name, row in loc.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['Start_GPS_Latitude'], row['Start_GPS_Longitude']],
        color="blue",
        tooltip=name, 
        fill=True
    ).add_to(start_map)
for name, row in d.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['GPS_Latitude'], row['GPS_Longitude']],
        color="crimson",
        tooltip=name,
        fill=False,
    ).add_to(start_map)
for name, row in r.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['Start_GPS_Latitude'], row['Start_GPS_Longitude']],
        color="green",
        tooltip=name, 
        fill=True
    ).add_to(start_map)
display(start_map)

NameError: name 'loc' is not defined